# IGR204 - Projet

In [1]:
import numpy as np
import pandas as pd

In [2]:
import dash_core_components
print(dash_core_components.__version__)

1.10.1


In [3]:
estimations = ["lower", "midpoint", "higher"]
years = [2015, 2020, 2040, 2060]

# 1. Chargement des données

Data:
* `total-pw`: total plastic waste (kg/year)  -> Quantité totale de déchets plastiques émis
* `pmw`: mismanaged plastic waste (kg/year)  -> Quantité totale de déchets plastiques mal recyclés<br />
-> Ces données sont disponibles pour 2015, 2020, 2040, 2060<br />
-> Ces données sont disponibles sous forme de fourchette lower, midpoint, higher (les colonnes commencent par `midpoint-` ...)<br />
-> Les données pmw sont données fixes pour 2015 et en prévisionnel pour 2020, 2040, 2060 selon 3 scénarios (A, B, C)

Il y a ensuite de nombreuses colonnes donnant des statistiques sur les pays.

In [4]:
df = pd.read_excel("./Data/igr204-data.xlsx")
print(df.shape)
df.head()

(187, 108)


,Country,Country code,ISO code,Continent,Region,midpoint-total-pw-2015,midpoint-total-pw-2020,midpoint-total-pw-2040,midpoint-total-pw-2060,midpoint-mpw-2015,...,Oil - production(bbl/day),Oil - proved reserves(bbl),Population,Public debt(% of GDP),Railways(km),Reserves of foreign exchange & gold,Telephones - main lines in use,Telephones - mobile cellular,Total fertility rate(children born/woman),Unemployment rate(%)
0,Afghanistan,4,AFG,Asia,Southern Asia,437000000,524000000,928000000,1300000000,4.277045e+08,...,0.0,0.000000e+00,29928987.0,NaN,NaN,NaN,33100.0,15000.0,6.75,NaN
1,Albania,8,ALB,Europe,Southern Europe,81400000,92300000,117000000,126000000,8.140000e+07,...,2000.0,1.855000e+08,3563112.0,NaN,447.0,1.206000e+09,255000.0,1100000.0,2.04,14.8
2,Algeria,12,DZA,Africa,Northern Africa,1050000000,1220000000,1940000000,2650000000,8.400000e+08,...,1200000.0,1.187000e+10,32531853.0,37.4,3973.0,4.355000e+10,2199600.0,1447310.0,1.92,25.4
3,Angola,24,AGO,Africa,Middle Africa,208000000,255000000,785000000,1710000000,1.872000e+08,...,980000.0,2.288000e+10,11190786.0,NaN,2761.0,8.000000e+08,96300.0,130000.0,6.27,NaN
4,Antigua and Barbuda,28,ATG,Latin America and Caribbean,Caribbean,4510000,4900000,6530000,7660000,1.294985e+06,...,0.0,NaN,68722.0,NaN,NaN,NaN,38000.0,38200.0,2.26,11.0


In [6]:
df["Continent"].unique()

array(['Asia', 'Europe', 'Africa', 'Latin America and Caribbean',
       'Oceania', 'Northern America'], dtype=object)

In [4]:
df.columns.to_list()

['Country',
 'Country code',
 'ISO code',
 'Continent',
 'Region',
 'midpoint-total-pw-2015',
 'midpoint-total-pw-2020',
 'midpoint-total-pw-2040',
 'midpoint-total-pw-2060',
 'midpoint-mpw-2015',
 'midpoint-mpw-scenarioA-2020',
 'midpoint-mpw-scenarioA-2040',
 'midpoint-mpw-scenarioA-2060',
 'midpoint-mpw-scenarioB-2020',
 'midpoint-mpw-scenarioB-2040',
 'midpoint-mpw-scenarioB-2060',
 'midpoint-mpw-scenarioC-2020',
 'midpoint-mpw-scenarioC-2040',
 'midpoint-mpw-scenarioC-2060',
 'lower-total-pw-2015',
 'lower-total-pw-2020',
 'lower-total-pw-2040',
 'lower-total-pw-2060',
 'lower-mpw-2015',
 'lower-mpw-scenarioA-2020',
 'lower-mpw-scenarioA-2040',
 'lower-mpw-scenarioA-2060',
 'lower-mpw-scenarioB-2020',
 'lower-mpw-scenarioB-2040',
 'lower-mpw-scenarioB-2060',
 'lower-mpw-scenarioC-2020',
 'lower-mpw-scenarioC-2040',
 'lower-mpw-scenarioC-2060',
 'higher-total-pw-2015',
 'higher-total-pw-2020',
 'higher-total-pw-2040',
 'higher-total-pw-2060',
 'higher-mpw-2015',
 'higher-mpw-scenar

In [12]:
print(df.shape)
for estimation in estimations:
    df[estimation + "-percent-2015"] = df[estimation + "-mpw-2015"] / df[estimation + "-total-pw-2015"]
    for year in years[1:]:
        df[estimation + "-percent-" + str(year)] = df[estimation + "-mpw-scenarioB-" + str(year)] / df[estimation + "-total-pw-" + str(year)]
df['percent'] = df["midpoint-percent-2015"]

(187, 109)


# 2. Figures

## 2.1. Carte

In [13]:
import plotly.graph_objects as go


fig = go.Figure(data=go.Choropleth(
    locations = df['ISO code'],
    z = df['percent']*100,
    text = df['Country'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = '% mismanaged plastic',
))

fig.update_layout(
    title_text='2015 Mismanaged plastic proportion (midpoint)',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

## 2.2. Scatter plot

In [14]:
fig = go.Figure(data=go.Scatter(x=df["percent"], y=df["Population"], mode='markers'))
fig.show()

## 2.3. Evolution